# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [62]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import nltk
nltk.download(['punkt', 'wordnet'])
from nltk.tokenize import word_tokenize
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import re
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
import pickle

# magic word for producing visualizations in notebook
%matplotlib inline

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [35]:
# load data from database
def load_data():
    engine = create_engine('sqlite:///DisasterResponse.db')
    df =  pd.read_sql_table('DisasterResponse', engine)
    X = df.message.values
    y = df.iloc[:,5:]
    return X, y

In [36]:
X, y = load_data()

### 2. Write a tokenization function to process your text data

In [37]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

In [38]:
def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [39]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [41]:
# train classifier
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [42]:
def display_results(cv, y_test, y_pred):
    print(classification_report(y_test, y_pred, target_names=y_test.keys()))

In [43]:
# predict on test data
y_pred = pipeline.predict(X_test)

In [44]:
# display results
display_results(pipeline, y_test, y_pred)

                        precision    recall  f1-score   support

               request       0.84      0.40      0.54      1332
                 offer       0.00      0.00      0.00        36
           aid_related       0.75      0.52      0.61      3219
          medical_help       0.51      0.06      0.10       638
      medical_products       0.69      0.07      0.13       418
     search_and_rescue       0.50      0.04      0.07       192
              security       0.00      0.00      0.00       144
              military       0.44      0.08      0.14       245
           child_alone       0.00      0.00      0.00         0
                 water       0.77      0.23      0.35       500
                  food       0.85      0.38      0.52       878
               shelter       0.80      0.25      0.39       705
              clothing       0.64      0.08      0.14       115
                 money       0.70      0.04      0.08       170
        missing_people       1.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [45]:
parameters = {  
        'clf__estimator__min_samples_split': [2, 4],
        'clf__estimator__max_features': ['log2', 'auto']
}

cv = GridSearchCV(estimator=pipeline, param_grid=parameters)

In [46]:
cv.fit(X_train,y_train)    

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'clf__estimator__min_samples_split': [2, 4], 'clf__estimator__max_features': ['log2', 'auto']},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [47]:
clf = cv.predict(X_test)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [48]:
# display results
display_results(clf, y_test, y_pred)

                        precision    recall  f1-score   support

               request       0.84      0.40      0.54      1332
                 offer       0.00      0.00      0.00        36
           aid_related       0.75      0.52      0.61      3219
          medical_help       0.51      0.06      0.10       638
      medical_products       0.69      0.07      0.13       418
     search_and_rescue       0.50      0.04      0.07       192
              security       0.00      0.00      0.00       144
              military       0.44      0.08      0.14       245
           child_alone       0.00      0.00      0.00         0
                 water       0.77      0.23      0.35       500
                  food       0.85      0.38      0.52       878
               shelter       0.80      0.25      0.39       705
              clothing       0.64      0.08      0.14       115
                 money       0.70      0.04      0.08       170
        missing_people       1.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [57]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(KNeighborsClassifier(n_neighbors=20)))
])

In [58]:
# train classifier
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ic_params=None, n_jobs=1, n_neighbors=20, p=2,
           weights='uniform'),
           n_jobs=1))])

In [59]:
# predict on test data
y_pred = pipeline.predict(X_test)

In [60]:
# display results
display_results(pipeline, y_test, y_pred)

                        precision    recall  f1-score   support

               request       0.78      0.50      0.61      1332
                 offer       0.00      0.00      0.00        36
           aid_related       0.65      0.61      0.63      3219
          medical_help       0.35      0.14      0.20       638
      medical_products       0.53      0.12      0.19       418
     search_and_rescue       1.00      0.01      0.02       192
              security       0.00      0.00      0.00       144
              military       0.88      0.03      0.06       245
           child_alone       0.00      0.00      0.00         0
                 water       0.60      0.24      0.35       500
                  food       0.58      0.41      0.48       878
               shelter       0.45      0.24      0.31       705
              clothing       0.74      0.12      0.21       115
                 money       0.62      0.03      0.06       170
        missing_people       0.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 9. Export your model as a pickle file

In [63]:
# save the model to disk
filename = 'finalized_model.sav'
pickle.dump(clf, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.